### Generating Datasets

In [ ]:
import datasetCreation
import os
from tensorflow.python.client import device_lib
import tensorflow as tf

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

#Run from server

gpu7 = "GPU-71b4cdfc-e381-0b98-9b24-4fc06284b496" 
gpu5 = "GPU-99d0769a-9f86-4800-a40e-2320dddcf5d1" 
gpu6 = "GPU-7423cfb5-cff4-ec4d-7e96-ea6e1591d56f"
gpu4 = "GPU-c0a8738f-6dd0-1b78-c38f-4969fd3886a8"

os.environ["CUDA_VISIBLE_DEVICES"]= gpu7# + "," + gpu6
print(get_available_devices())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
path_directory = os.path.join("..","data","MICCAI_BraTS2020_TrainingData")

dataset_directory_t1 = os.path.join("..","data", 't1_images.hdf5')
dataset_directory_t1ce = os.path.join("..","data", 't1ce_images.hdf5')
dataset_directory_t2 = os.path.join("..","data", 't2_images.hdf5')
dataset_directory_flair = os.path.join("..","data", 'flair_images.hdf5')
dataset_directory_seg = os.path.join("..","data", 'seg_images.hdf5')
dataset_directory_weight_map = os.path.join("..","data", 'weight_map.hdf5')

In [ ]:
"""
command for creating dataset with images
"""

datasetCreation.create_dataset(path_directory, dataset_directory_t1, features = True)
datasetCreation.create_dataset(path_directory, dataset_directory_t1ce, features = True)
datasetCreation.create_dataset(path_directory, dataset_directory_t2, features = True)
datasetCreation.create_dataset(path_directory, dataset_directory_flair, features = True)

In [ ]:
datasetCreation.create_dataset(path_directory, dataset_directory_seg, features = False)

creating weight maps

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt  # for visualising and debugging
from scipy.ndimage.morphology import distance_transform_edt
from skimage.io import imsave, imread
from skimage.segmentation import find_boundaries
from helperFunctions import visualize, denormalize, visualize_histories, load_images_from_hdf5

#from demos.unet.file_sorter import get_user_args

W_0, SIGMA = 2, 1

def construct_weights_and_mask(img):
    seg_boundaries = find_boundaries(img, mode='inner')

    bin_img = img > 0
    # take segmentations, ignore boundaries
    binary_with_borders = np.bitwise_xor(bin_img, seg_boundaries)

    foreground_weight = 1 - binary_with_borders.sum() / binary_with_borders.size
    background_weight = 1 - foreground_weight

    # build euclidean distances maps for each cell:
    cell_ids = [x for x in np.unique(img) if x > 0]
    distances = np.zeros((img.shape[0], img.shape[1], len(cell_ids)))

    for i, cell_id in enumerate(cell_ids):
        distances[..., i] = distance_transform_edt(img != cell_id)

    # we need to look at the two smallest distances
    distances.sort(axis=-1)

    
    if np.shape(distances)[2] == 0 or np.shape(distances)[2] == 1:
        weight_map = background_weight

    else:
        
        weight_map = W_0 * np.exp(-(1 / (2 * SIGMA ** 2)) * ((distances[..., 0] + distances[..., 1]) ** 2))
        weight_map[binary_with_borders] = foreground_weight
        weight_map[~binary_with_borders] += background_weight
    
    #return weight_map, binary_with_borders
    return weight_map

#weight_map, binary_with_borders = construct_weights_and_mask(train_labels[...,100])

In [ ]:
def assign_weights_to_images(image_labels, weights):
    """
    create an image with weights according to the labels indicated
    """ 
    labels = range(0,5)
    new_img = np.zeros(np.shape(image_labels))
    for i in range(len(new_img)):
        for j in range(len(new_img)):
            for k in labels:
                if image_labels[i,j] == k:
                    new_img[i,j] = weights[k]
                    break

    return new_img

In [ ]:
new_img = assign_weights_to_images(train_labels[...,100], weights)

In [ ]:
np.max(final_weights)

In [ ]:
plt.imshow(final_weights)
plt.colorbar()

In [ ]:
print(np.shape(weight_map))
plt.imshow(weight_map[...,100])
plt.imshow(train_labels[...,100])
plt.colorbar()

In [ ]:
train_begin = 0
train_stop = 369*155
n_images = train_stop
#weight_map_dist = np.zeros((240,240,n_images))
#weight_map_personal = np.zeros((240,240,n_images))
final_weight_map = np.zeros((240,240,n_images))
weights = [0.1,1.8,1.2,0.01,1.2]

train_labels, valid_labels = load_images_from_hdf5(dataset_directory_seg,train_begin,train_stop,n_images)

In [ ]:
for i in range(n_images):
    weight_map_dist = construct_weights_and_mask(train_labels[...,i])
    weight_map_personal = assign_weights_to_images(train_labels[...,i], weights)
    final_weight_map[...,i] = np.add(weight_map_dist, weight_map_personal)

In [ ]:
np.shape(final_weight_map)

In [ ]:
from helperFunctions import visualize
for i in range(549,550):
    visualize(
        labels = train_labels[...,i],
        weight_map_distances = construct_weights_and_mask(train_labels[...,i]),
        weight_map_personal = assign_weights_to_images(train_labels[...,i], weights),
        final_weight_map = final_weight_map[...,i]
    )

In [ ]:
plt.imshow(final_weight_map[...,550])
plt.colorbar()

In [ ]:
np.max(np.add(weight_map_dist[...,540], weight_map_personal[...,540]))

In [ ]:
test = assign_weights_to_images(train_labels[...,550],weights)
plt.imshow(test)
np.max(test)
np.max(weight_map_personal[...,550])

In [ ]:
"""
command for creating dataset with weightmaps
"""
datasetCreation.create_h5py(final_weight_map, dataset_directory_weight_map)